In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 6.8 MB/s 
     |████████████████████████████████| 6.6 MB 67.8 MB/s 
     |████████████████████████████████| 120 kB 93.5 MB/s 


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 8.7 MB/s 
     |████████████████████████████████| 212 kB 99.0 MB/s 
     |████████████████████████████████| 115 kB 74.9 MB/s 
     |████████████████████████████████| 127 kB 92.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 6.1 MB/s 


In [5]:
!nvidia-smi

Thu Sep  8 12:08:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import os
import gc
import math
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')
from tqdm import tqdm
import re

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam, SGD, AdamW, RAdam
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import StratifiedKFold,StratifiedGroupKFold,GroupKFold
from sklearn.metrics import log_loss,f1_score

from transformers import AutoModel, AutoConfig, AutoTokenizer, AdamW, DataCollatorWithPadding
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
DIR = '/content/drive/MyDrive/Competitions/Signate/MUFJ'
INPUT_DIR = os.path.join(DIR,'input')
OUTPUT_DIR = os.path.join(DIR,'output')
OUTPUT_SUB_DIR = os.path.join(OUTPUT_DIR,'submission')
#OUTPUT_MODEL_DIR = os.path.join(OUTPUT_DIR,'model')
OUTPUT_MODEL_DIR = DIR + '/output/model/EXP08/'
if not os.path.exists(OUTPUT_MODEL_DIR):
    os.makedirs(OUTPUT_MODEL_DIR)

In [8]:
class CFG:
    wandb = False
    apex = True
    model = 'microsoft/deberta-v3-large'
    seed = 42
    n_splits = 4
    max_len = 256
    dropout = 0.2
    target_size=1
    n_accumulate=1
    print_freq = 50
    min_lr=1e-6
    scheduler = 'cosine' #'linear','cosine'
    batch_size = 16
    num_workers = 2
    lr = 2e-5
    weigth_decay = 0.01
    epochs = 4
    n_fold = 4
    trn_fold = [0, 1, 2, 3]
    train = True 
    num_warmup_steps = 0
    num_cycles=0.5
    debug = False
    gradient_checkpointing = True
    freezing = True
    #loss_fn = 'focal_loss'
    #alpha = 0.0
    #gamma = 1
    #logits=True
    #weights=[1, 4, 1, 1]
    optimizer = 'AdamW'
    reinit_layer=5
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)

if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0, 1]

In [9]:
# Loss Func
class ClassificationFocalLoss(nn.Module):
    def __init__(self, n_classes: int, alpha=0.3, gamma=2, weights = None, logits=False):
        """
        :param n_classes: 
        :param gamma: 簡単なサンプルの重み. 大きいほど簡単なサンプルを重視しない.
        :param weights: weights by classes,
        :param logits:
        """
        super().__init__()
        self._alpha = alpha
        self._n_classes = n_classes
        self._noise_val = alpha / n_classes
        self.gamma = gamma
        self.class_weight_tensor = torch.tensor(weights).view(-1, ).to(device) if weights else None
        self.logits = logits
        if not logits and weights is not None:
            RuntimeWarning("重みを適用するにはlogitsをTrueにしてください.")

    def forward(self, pred: torch.Tensor, teacher: torch.Tensor) -> float:
        """
        :param pred: batch_size, n_classes
        :param teacher: batch_size, 
        :return: 
        """
        if self.logits:
            if teacher.ndim == 1:  # 1次元ならonehotの2次元tensorにする
                teacher = torch.eye(self._n_classes)[teacher]
            teacher = teacher * (1 - self._alpha) + self._noise_val
            ce_loss = F.binary_cross_entropy_with_logits(pred, teacher, reduce=False)
            pt = torch.exp(-ce_loss)

            if not self.class_weight_tensor == None:
                class_weight_tensor = self.class_weight_tensor.expand(pred.shape[0],
                                                                      self.class_weight_tensor.shape[0], )
                focal_loss = (1. - pt) ** self.gamma * (ce_loss * class_weight_tensor)
            else:
                focal_loss = (1. - pt) ** self.gamma * ce_loss
        else:
            # if teacher.ndim == 2:  # onehotの2次元tensorなら1次元にする
            #     teacher = teacher.argmax(1).to(torch.long)
            teacher = teacher * (1 - self._alpha) + self._noise_val
            ce_loss = F.cross_entropy(pred, teacher, reduce=False)
            pt = torch.exp(-ce_loss)
            focal_loss = (1. - pt) ** self.gamma * ce_loss
        return torch.mean(focal_loss)

def criterion(outputs, labels):
    return nn.BCEWithLogitsLoss()(outputs, labels)

def get_score(labels, outputs):
    thresh = 0.5
    y_pred = outputs
    y_true = labels
    return f1_score(y_true, (y_pred>thresh).astype(int))


def get_logger(filename=OUTPUT_MODEL_DIR+'train'):
    from logging import getLogger, INFO, FileHandler, Formatter, StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=CFG.seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [10]:
train = pd.read_csv(os.path.join(INPUT_DIR,'train.csv'))
test = pd.read_csv(os.path.join(INPUT_DIR,'test.csv'))
sub = pd.read_csv(os.path.join(INPUT_DIR,'sample_submit.csv'),header=None)
sub.columns = ['id','state']

In [11]:
def create_ctg2(df):
  group = df.groupby(['country', 'category1']).agg({"category2":list}).reset_index()
  group["category2"] = group["category2"].map(lambda s: "; ".join(s))
  group.rename(columns={"category2":"country_category1"}, inplace=True)
  df = df.merge(group, how='left', on=["category1", "country"])
  return df

In [12]:
def freeze(module):
    """
    Freezes module's parameters.
    """
    
    for parameter in module.parameters():
        parameter.requires_grad = False
        
def get_freezed_parameters(module):
    """
    Returns names of freezed parameters of the given module.
    """
    
    freezed_parameters = []
    for name, parameter in module.named_parameters():
        if not parameter.requires_grad:
            freezed_parameters.append(name)
            
    return freezed_parameters

def set_embedding_parameters_bits(embeddings_path, optim_bits=32):
    """
    https://github.com/huggingface/transformers/issues/14819#issuecomment-1003427930
    """
    
    embedding_types = ("word", "position", "token_type")
    for embedding_type in embedding_types:
        attr_name = f"{embedding_type}_embeddings"
        
        if hasattr(embeddings_path, attr_name): 
            bnb.optim.GlobalOptimManager.get_instance().register_module_override(
                getattr(embeddings_path, attr_name), 'weight', {'optim_bits': optim_bits}
            )

In [13]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub('',text)

def remove_html(text):
    html=re.compile(r"<[^>]*?>")
    return html.sub('',text)

def cleaning(texts):
    clean_texts = []
    for text in texts:
        # htmlタグを削除
        text = remove_URL(text)
        text = remove_html(text)
        #アルファベット以外をスペースに置き換え
        #clean_punc = re.sub(r'[^a-zA-Z]', ' ', text)
        #改行削除
        text = text.replace("\n","")
        clean_texts.append(text)
    return clean_texts


from text_unidecode import unidecode
from typing import Dict, List, Tuple
import codecs

def replace_encoding_with_utf8(error: UnicodeError) -> Tuple[bytes, int]:
    return error.object[error.start : error.end].encode("utf-8"), error.end


def replace_decoding_with_cp1252(error: UnicodeError) -> Tuple[str, int]:
    return error.object[error.start : error.end].decode("cp1252"), error.end

# Register the encoding and decoding error handlers for `utf-8` and `cp1252`.
codecs.register_error("replace_encoding_with_utf8", replace_encoding_with_utf8)
codecs.register_error("replace_decoding_with_cp1252", replace_decoding_with_cp1252)

def resolve_encodings_and_normalize(text: str) -> str:
    """Resolve the encoding problems and normalize the abnormal characters."""
    text = (
        text.encode("raw_unicode_escape")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
        .encode("cp1252", errors="replace_encoding_with_utf8")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
    )
    text = unidecode(text)
    return text


train['html_content'] = cleaning(train['html_content'])
#train = create_ctg2(train)
train['html_content'] = train['html_content'].apply(lambda x : resolve_encodings_and_normalize(x))
train['html_content'] = train['html_content'].fillna('missing')
#train['inputs'] = train.goal + '  ' + train.duration.astype(str) + ' [SEP] ' + train.country + ' ; ' + train.category1 + ' ; ' + train.country_category1 + ' [SEP] ' + train.html_content
train['inputs'] = train.goal + ' ; ' + train.duration.astype(str) + ' [SEP] ' + train.country + ' [SEP] ' + train.category1 + ' ; ' + train.category2 + ' [SEP] ' + train.html_content
train = train.rename(columns = {"state": "label"})

In [14]:
skf = StratifiedKFold(n_splits=CFG.n_splits,shuffle=True,random_state=CFG.seed)
for fold, ( _, val_) in enumerate(skf.split(train, train.label)):
    train.loc[val_ , "kfold"] = int(fold)
    
train["kfold"] = train["kfold"].astype(int)

if CFG.debug:
    display(train.groupby('kfold').size())
    train = train.sample(n=1000, random_state=0).reset_index(drop=True)
    display(train.groupby('kfold').size())

In [15]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model)
#tokenizer.add_tokens([f"\n"], special_tokens=True)
tokenizer.save_pretrained(OUTPUT_MODEL_DIR+'tokenizer/')
CFG.tokenizer = tokenizer
SEP = tokenizer.sep_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
class Dataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = CFG.max_len
        self.text = df['inputs'].values
        self.tokenizer = CFG.tokenizer
        self.targets = df['label'].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        text = self.text[index]
        inputs = tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length = self.max_len
        )
        samples = {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask'],
            'target': self.targets[index]
        }

        if 'token_type_ids' in inputs:
            samples['token_type_ids'] = inputs['token_type_ids']
            
        return samples

In [17]:
class Collate:
    def __init__(self, tokenizer, isTrain=True):
        self.tokenizer = tokenizer
        self.isTrain = isTrain
        # self.args = args

    def __call__(self, batch):
        output = dict()
        output["input_ids"] = [sample["input_ids"] for sample in batch]
        output["attention_mask"] = [sample["attention_mask"] for sample in batch]
        if self.isTrain:
            output["target"] = [sample["target"] for sample in batch]

        # calculate max token length of this batch
        batch_max = max([len(ids) for ids in output["input_ids"]])

        # add padding
        if self.tokenizer.padding_side == "right":
            output["input_ids"] = [s + (batch_max - len(s)) * [self.tokenizer.pad_token_id] for s in output["input_ids"]]
            output["attention_mask"] = [s + (batch_max - len(s)) * [0] for s in output["attention_mask"]]
        else:
            output["input_ids"] = [(batch_max - len(s)) * [self.tokenizer.pad_token_id] + s for s in output["input_ids"]]
            output["attention_mask"] = [(batch_max - len(s)) * [0] + s for s in output["attention_mask"]]

        # convert to tensors
        output["input_ids"] = torch.tensor(output["input_ids"], dtype=torch.long)
        output["attention_mask"] = torch.tensor(output["attention_mask"], dtype=torch.long)
        if self.isTrain:
            output["target"] = torch.tensor(output["target"], dtype=torch.long)

        return output
    
collate_fn = Collate(CFG.tokenizer, isTrain=True)
#collate_fn = DataCollatorWithPadding(tokenizer=tokenizer)

In [18]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9) #
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

class MaxPooling(nn.Module):
    def __init__(self):
        super(MaxPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        embeddings = last_hidden_state.clone()
        embeddings[input_mask_expanded == 0] = -1e4
        max_embeddings, _ = torch.max(embeddings, dim=1)
        return max_embeddings

In [19]:
# ====================================================
# Model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, model_name):
        super(CustomModel, self).__init__()
        # Header (fast or normal)

        hidden_dropout_prob: float = 0.15

        self.config = AutoConfig.from_pretrained(model_name)

        self.model = AutoModel.from_pretrained(model_name)

        self.config.update(
            {
                "output_hidden_states": True,
                "hidden_dropout_prob": hidden_dropout_prob,
                "num_labels": CFG.target_size,
            }
        )

        self.dropout = nn.Dropout(self.config.hidden_dropout_prob)
        self.mean_pooler = MeanPooling()
        self.max_pooler = MaxPooling()
        self.output = nn.Linear(self.config.hidden_size*2, CFG.target_size)
        self._init_weights(self.output)
        self.layer_norm1 = nn.LayerNorm(self.config.hidden_size*2)

        
        # Gradient_checkpointing
        if CFG.gradient_checkpointing:
            (self.model).gradient_checkpointing_enable()
        
        # Freezing
        if CFG.freezing:
            # freezing embeddings and first 2 layers of encoder
            freeze((self.model).embeddings)
            freeze((self.model).encoder.layer[:2])
            CFG.after_freezed_parameters = filter(lambda parameter: parameter.requires_grad, (self.model).parameters())
        
        

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def feature(self, ids, mask):
        output = self.model(input_ids=ids, 
                         attention_mask=mask,
                         output_hidden_states=False)
        last_hidden_states = output[0]
        #feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature
    
    def forward(self, ids, mask):

        #feature = self.feature(ids, mask)
        transformer_out = self.model(input_ids=ids, 
                         attention_mask=mask,
                         output_hidden_states=False)
        output = transformer_out.last_hidden_state
        output = self.dropout(output)
      
        mean_pool = self.mean_pooler(output, mask)
        max_pool = self.max_pooler(output,mask)

        concat = torch.cat([mean_pool,max_pool],dim=1)

        sequence_output = self.layer_norm1(concat)

        logits = self.output(sequence_output)
    
        return logits

In [20]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def asMinutes(s):
    m = math.floor(s/60)
    s -= m * 60
    return "%dm %ds" % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return "%s (remain %s)" % (asMinutes(s), asMinutes(rs))

def get_scheduler(cfg, optimizer, num_train_steps):
    if cfg.scheduler == 'linear':
        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
        )
    elif cfg.scheduler == 'cosine':
        scheduler = get_cosine_schedule_with_warmup(
            optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
        )
    return scheduler

def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()

    dataset_size = 0
    running_loss = 0

    start = end = time.time()

    for step, data in enumerate(dataloader):
        ids = data['input_ids'].to(device)
        mask = data['attention_mask'].to(device)
        targets = data['target'].to(device)

        batch_size = ids.size(0)
        outputs = model(ids, mask)
        loss = criterion(outputs.view(-1, 1).float(), targets.view(-1, 1).float())

        #accumulate
        loss = loss / CFG.n_accumulate
        loss.backward()
        if (step +1) % CFG.n_accumulate == 0:
            optimizer.step()

            optimizer.zero_grad()
            if scheduler is not None:
                scheduler.step()
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size

        end = time.time()
        
        if step % CFG.print_freq == 0 or step == (len(dataloader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  .format(epoch+1, step, len(dataloader), 
                          remain=timeSince(start, float(step+1)/len(dataloader))))

    gc.collect()

    return epoch_loss


@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()

    dataset_size = 0
    running_loss = 0

    start = end = time.time()
    pred = []

    for step, data in enumerate(dataloader):
        ids = data['input_ids'].to(device)
        mask = data['attention_mask'].to(device)
        targets = data['target'].to(device)

        batch_size = ids.size(0)
        outputs = model(ids, mask)
        loss = criterion(outputs.view(-1, 1).float(), targets.view(-1, 1).float())
        pred.append(outputs.sigmoid().to('cpu').numpy())

        running_loss += (loss.item()* batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size

        end = time.time()

        if step % CFG.print_freq == 0 or step == (len(dataloader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  .format(step, len(dataloader),
                          remain=timeSince(start, float(step+1)/len(dataloader))))
            
    pred = np.concatenate(pred)
            
    return epoch_loss, pred

In [21]:
def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
          'lr': encoder_lr, 'weight_decay': weight_decay},
        {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
          'lr': encoder_lr, 'weight_decay': 0.0},
        {'params': [p for n, p in model.named_parameters() if "model" not in n],
          'lr': decoder_lr, 'weight_decay': 0.0}
    ]
    return optimizer_parameters


def train_loop(fold):
    #wandb.watch(model, log_freq=100)

    LOGGER.info(f'-------------fold:{fold} training-------------')

    train_data = train[train.kfold != fold].reset_index(drop=True)
    valid_data = train[train.kfold == fold].reset_index(drop=True)
    valid_labels = valid_data.label.values

    trainDataset = Dataset(train_data, CFG.tokenizer, CFG.max_len)
    validDataset = Dataset(valid_data, CFG.tokenizer, CFG.max_len)

    train_loader = DataLoader(trainDataset,
                              batch_size = CFG.batch_size,
                              shuffle=True,
                              collate_fn = collate_fn,
                              num_workers = CFG.num_workers,
                              pin_memory = True,
                              drop_last=True)
    
    valid_loader = DataLoader(validDataset,
                              batch_size = CFG.batch_size*2,
                              shuffle=False,
                              collate_fn = collate_fn,
                              num_workers = CFG.num_workers,
                              pin_memory = True,
                              drop_last=False)
    
    model = CustomModel(CFG.model)
    torch.save(model.config, OUTPUT_MODEL_DIR+'config.pth')
    model.to(device)
    optimizer_parameters = get_optimizer_params(model=model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weigth_decay,
                                                )
    
    if CFG.optimizer == 'AdamW':
        optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
        #optimizer = AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weigth_decay)
    if CFG.optimizer == 'RAdam':
        optimizer = RAdam(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
        #optimizer = RAdam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weigth_decay)
    num_train_steps = int(len(train_data) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # loop
    best_score = 0

    for epoch in range(CFG.epochs):
        start_time = time.time()

        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, train_loader, device, epoch)
        valid_epoch_loss, pred = valid_one_epoch(model, valid_loader, device, epoch)

        score = get_score(valid_labels,pred)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {train_epoch_loss:.4f}  avg_val_loss: {valid_epoch_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}')

            
        if score > best_score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': pred},
                        OUTPUT_MODEL_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")
            
    predictions = torch.load(OUTPUT_MODEL_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']
    valid_data['pred'] = predictions
    
    
    temp = valid_data['pred'].values
    print(get_score(valid_data['label'].values,temp))

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_data

In [22]:
if __name__ == '__main__':
    
    def get_result(oof_df):
        labels = oof_df['label'].values
        preds = oof_df['pred'].values
        score = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.4f}')
    
    if CFG.train:
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
            
        oof_df = oof_df.reset_index(drop=True)
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        oof_df.to_pickle(OUTPUT_MODEL_DIR+'oof_df.pkl')
        oof_df.to_csv(OUTPUT_MODEL_DIR+f'oof_df.csv', index=False)

-------------fold:0 training-------------
INFO:__main__:-------------fold:0 training-------------


Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: [1][0/458] Elapsed 0m 7s (remain 54m 44s) 
Epoch: [1][50/458] Elapsed 1m 56s (remain 15m 31s) 
Epoch: [1][100/458] Elapsed 3m 45s (remain 13m 18s) 
Epoch: [1][150/458] Elapsed 5m 35s (remain 11m 21s) 
Epoch: [1][200/458] Elapsed 7m 24s (remain 9m 28s) 
Epoch: [1][250/458] Elapsed 9m 14s (remain 7m 37s) 
Epoch: [1][300/458] Elapsed 11m 3s (remain 5m 46s) 
Epoch: [1][350/458] Elapsed 12m 53s (remain 3m 55s) 
Epoch: [1][400/458] Elapsed 14m 43s (remain 2m 5s) 
Epoch: [1][450/458] Elapsed 16m 32s (remain 0m 15s) 
Epoch: [1][457/458] Elapsed 16m 48s (remain 0m 0s) 
EVAL: [0/77] Elapsed 0m 1s (remain 1m 54s) 
EVAL: [50/77] Elapsed 0m 53s (remain 0m 27s) 


Epoch 1 - avg_train_loss: 0.5232  avg_val_loss: 0.5979  time: 1089s
INFO:__main__:Epoch 1 - avg_train_loss: 0.5232  avg_val_loss: 0.5979  time: 1089s
Epoch 1 - Score: 0.6304
INFO:__main__:Epoch 1 - Score: 0.6304
Epoch 1 - Save Best Score: 0.6304 Model
INFO:__main__:Epoch 1 - Save Best Score: 0.6304 Model


EVAL: [76/77] Elapsed 1m 20s (remain 0m 0s) 
Epoch: [2][0/458] Elapsed 0m 2s (remain 19m 16s) 
Epoch: [2][50/458] Elapsed 1m 52s (remain 14m 55s) 
Epoch: [2][100/458] Elapsed 3m 41s (remain 13m 3s) 
Epoch: [2][150/458] Elapsed 5m 31s (remain 11m 13s) 
Epoch: [2][200/458] Elapsed 7m 20s (remain 9m 23s) 
Epoch: [2][250/458] Elapsed 9m 10s (remain 7m 34s) 
Epoch: [2][300/458] Elapsed 11m 0s (remain 5m 44s) 
Epoch: [2][350/458] Elapsed 12m 49s (remain 3m 54s) 
Epoch: [2][400/458] Elapsed 14m 39s (remain 2m 4s) 
Epoch: [2][450/458] Elapsed 16m 28s (remain 0m 15s) 
Epoch: [2][457/458] Elapsed 16m 44s (remain 0m 0s) 
EVAL: [0/77] Elapsed 0m 1s (remain 1m 53s) 
EVAL: [50/77] Elapsed 0m 53s (remain 0m 27s) 


...

Epoch 4 - avg_train_loss: 0.0659  avg_val_loss: 0.5109  time: 1073s
INFO:__main__:Epoch 4 - avg_train_loss: 0.0659  avg_val_loss: 0.5109  time: 1073s
Epoch 4 - Score: 0.8203
INFO:__main__:Epoch 4 - Score: 0.8203


EVAL: [76/77] Elapsed 1m 18s (remain 0m 0s) 


========== fold: 3 result ==========
INFO:__main__:========== fold: 3 result ==========
Score: 0.8222
INFO:__main__:Score: 0.8222
========== CV ==========
INFO:__main__:========== CV ==========


0.8222133439872154


Score: 0.8155
INFO:__main__:Score: 0.8155


In [23]:
A = pd.read_csv(OUTPUT_MODEL_DIR+'oof_df.csv')
A.head()

,id,goal,country,duration,category1,category2,html_content,label,inputs,kfold,pred
0,train_00002,2001-3000,US,38,art,performance art,"I want to perform this piece guerilla style, o...",0,2001-3000 ; 38 [SEP] US [SEP] art ; performanc...,0,0.816360
1,train_00006,2001-3000,CA,30,music,classical music,The Crimson String Quartet (CSQ) is returning ...,1,2001-3000 ; 30 [SEP] CA [SEP] music ; classica...,0,0.742539
2,train_00016,3001-4000,GB,30,journalism,web,I am interested in setting up a small news web...,0,3001-4000 ; 30 [SEP] GB [SEP] journalism ; web...,0,0.001581
3,train_00017,11001-12000,US,30,food,drinks,Why Schenk-Atwood?Our neighborhood has just ab...,1,11001-12000 ; 30 [SEP] US [SEP] food ; drinks ...,0,0.540023
4,train_00018,12001-13000,US,28,journalism,web,Mega Visions app has been approved on all majo...,1,12001-13000 ; 28 [SEP] US [SEP] journalism ; w...,0,0.840174
